# Plots
For the plots in section 5: "Results and dicsuccsion".
## Physical properties of fluids and define functions
Define the used functions and the the physical properties of two working fluids: water and 50% Glycerin-water mixture.

Define the used functions to calculate the dynamic contact angles based on five models: MKT, HDT, combined model, Jiang, Bracke.

In [ ]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.pyplot import figure
import matplotlib.colors as mcolors
import itertools

data_directory = 'Data/'
figure_directory = 'Figure/'

# Physical properties of water at 20°C
rho_water = 998 # water density in kg/m3
mu_water = 0.0010005 # dynamic viscosity in Pa*s
sigma_air_water = 72.74e-3

# Physical properties of 50% Glycerin-water mixture at 20°C
rho_glycerin = 1142
mu_glycerin = 0.008026
sigma_air_glycerin = 68.12e-3

cos = np.cos
tanh = np.tanh
arccos = np.arccos
arcsinh = np.arcsinh

# Function to calculate the capillary number of contact line
def cal_caCL(uCL, mu, sigma):
    caCL = mu * uCL / sigma
    return caCL

# Function to calculate the dynamic contact angle with MKT model, T=20°C
def cal_theta_MKT(uCL, mu, sigma, theta_0, lamda, kappa0):
    cos_theta_MKT = cos(theta_0) - 2*1.380649e-23*295.15 / (sigma*lamda**2) * arcsinh(uCL/(2*kappa0*lamda))
    theta_MKT = arccos(cos_theta_MKT)
    theta_MKT = np.degrees(theta_MKT)
    return theta_MKT

# Function to calculate the dynamic contact angle with HDT model
def cal_theta_HDT(uCL, mu, sigma, theta_e, beta):
    caCL =  uCL * mu / sigma
    theta_HDT = (theta_e**3 + 9*caCL*beta)**(1/3)
    theta_HDT = np.degrees(theta_HDT)
    return theta_HDT

# Function to calculate the dynamic contact angle with combined model
def cal_theta_combined(uCL, mu, sigma, theta_e, zeta, beta):
    caCL =  uCL * mu / sigma
    theta_m = arccos(-zeta*uCL/sigma + cos(theta_e))
    theta_voinov_cox = (theta_m**3 + 9*caCL*beta)**(1/3)
    theta_voinov_cox = np.degrees(theta_voinov_cox)
    return theta_voinov_cox

# Function to calculate the dynamic contact angle with Jiang model
def cal_theta_Jiang(theta_e, ca):
    cos_theta_app = cos(theta_e) - tanh(4.96 * ca ** 0.702)*(cos(theta_e)+1)
    theta_app = arccos(cos_theta_app)
    theta_app = np.degrees(theta_app)
    return theta_app

# Function to calculate the dynamic contact angle with Bracke model
def cal_theta_Bracke(theta_e, ca):
    cos_theta_app = cos(theta_e) - (2*ca**(1/2))*(cos(theta_e)+1)
    theta_app = arccos(cos_theta_app)
    theta_app = np.degrees(theta_app)
    return theta_app

def cal_errors(values):
    mean = []
    std_dev = []
    
    mean.append(np.mean(values))
    std_dev.append(np.std(values))
    return mean, std_dev

def create_tick_labels(channels, fluids):
    """
    For violin plots, create tick labels for x-axis. Each label is composed
    of the employed mesh resolutions and a solver name.
    """
    fluid_label = ""
    for fluid in fluids:
        fluid_label = fluid_label + str(fluid) + ' '
    fluid_label = fluid_label.replace(' ', '$\quad\quad$') + '\n'
    tick_labels = [fluid_label + channel for channel in channels]
    return tick_labels

def assign_styles(unique_values):
    """
    Assign each value in 'unique_values' a marker and line style.
    Returns mappings as two dictionaries.
    """
    unique_values.sort()

    markers = itertools.cycle(['o', 'x', '+', 's', 'd', '*', '^', 'v'])
    # Reserve solid line style for reference solutions
    lines = itertools.cycle(['dotted', 'dashed', 'dashdot', (0, (3, 1, 1, 1))])
    colors = itertools.cycle(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
                              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

    markerdict = dict()
    linestyledict = dict()
    colordict = dict()

    for value in unique_values:
        markerdict[value] = next(markers)
        linestyledict[value] = next(lines)
        colordict[value] = next(colors)

    return markerdict, linestyledict, colordict


from collections import OrderedDict

linestyles_dict = OrderedDict(
    [
        ("solid", (0, ())),
        ("loosely dotted", (0, (1, 10))),
        ("dotted", (0, (1, 5))),
        ("densely dotted", (0, (1, 1))),
        ("loosely dashed", (0, (5, 10))),
        ("dashed", (0, (5, 5))),
        ("densely dashed", (0, (5, 1))),
        ("loosely dashdotted", (0, (3, 10, 1, 10))),
        ("dashdotted", (0, (3, 5, 1, 5))),
        ("densely dashdotted", (0, (3, 1, 1, 1))),
        ("loosely dashdotdotted", (0, (3, 10, 1, 10, 1, 10))),
        ("dashdotdotted", (0, (3, 5, 1, 5, 1, 5))),
        ("densely dashdotdotted", (0, (3, 1, 1, 1, 1, 1))),
    ]
)

def creat_single_violing_plot(df, error_metrics, yscale='log',
                        ylabel='', file_name_addendum='', groupbyparameters=None):
    
    # Prepare color dictionaries for consistent coloring
    color_list_full = ['steelblue', 'red', 'mediumseagreen', 'gold', 'hotpink', 'sandybrown'] + \
                 [color for color in mcolors.CSS4_COLORS]
    darker_color_list_full = ['darkblue', 'darkred', 'darkgreen', 'darkgoldenrod', 'mediumvioletred', 'sienna'] + \
                        [color for color in mcolors.CSS4_COLORS][1:]

    channel_list = ([channel for channel, _ in df.groupby(level=["channel-fluid"])]) 
    color_list = color_list_full[:len(channel_list)] 
    darker_color_list = darker_color_list_full[:len(channel_list)]
    color_dict = {k:[v1,v2] for k,v1,v2 in zip(channel_list,color_list,darker_color_list)}
    
    tick_vector = []
    fluids = []
    ax = plt.axes() 
    
    for channel, channeldf in df.groupby(level=["channel-fluid"]):
        # Number of different resolutions by creating a set as intermediate step
        #nchannel = len(set(channeldf.index.get_level_values('fluid')))
        nchannel = 1
        tick_vector.append(channel)
        xpos = [x + nchannel*(len(tick_vector)-1) for x in range(1,nchannel+1)]
        #xpos = [len(tick_vector)]
        error_metrics.sort() # make sure the order is always the same
        for metric_count, error_metric in enumerate(error_metrics):
            plotcolumns = []
            plotcolumns.append(channeldf[error_metric])
            violin_parts = ax.violinplot(plotcolumns, positions=xpos, showmeans=True,
                                             widths=0.8, showmedians=False, showextrema=True)
            current_color = color_dict[channel][metric_count % 2] # modulus repeats colors for more than two metrics
            
            for partname in ('cbars','cmins','cmaxes','cmeans'):
                vp = violin_parts[partname]
                vp.set_edgecolor(current_color)
                vp.set_linewidth(1)
            '''
            vp = violin_parts['cmeans']
            vp.set_edgecolor(current_color)
            vp.set_linewidth(1)
            '''
            for pc in violin_parts['bodies']:
                pc.set_facecolor(current_color)
                pc.set_edgecolor(current_color)

        tick_positions = [nchannel*x+1 for x in range(len(tick_vector))]
        
        #print(tick_vector)
        tick_labels = create_tick_labels(tick_vector, fluids)
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_xticks(range(1, nchannel*len(tick_vector)+1), minor=True)
        #ax.grid(which="major")
        ax.set_ylabel(ylabel)

        if not os.path.isdir(figure_directory):
            os.mkdir(figure_directory)
        plt.savefig(os.path.join(figure_directory, "violin_" + file_name_addendum ), bbox_inches="tight")

## Violin plots for validation of automated image analysis
The violin plots for the validation of automated analysis in section 5.1:
- Interface detection (Fig. 10): `u_error` 
- Contact angle measurement (Fig. 11): `theta_error` 

In [ ]:
figure(figsize=(8, 3), dpi=300)

# Load the csv file
df_error = pd.read_csv(os.path.join(data_directory, "data_violin_plots.csv"), sep=';')

# Metadata columns
index_column = ['channel-fluid']
df_error.set_index(index_column, inplace=True)
df_error.sort_index(inplace=True)

# Data columns: "u_error" for velocity error, "theta_error" for dynamic contact angle error
data_column = ['theta_error']

# Error metrics for which plots are created
error_metrics = list(data_column)
groupbyviolin = index_column

y_label_u = r'Interface velocity error $|U_{theo}-U_{code}|/U_{theo}$'
y_label_theta = r'Contact angle error $|\theta_{manual}-\theta_{code}|/\theta_{manual}$'

creat_single_violing_plot(df_error, error_metrics, yscale='linear',
                    ylabel=y_label_theta, file_name_addendum='theta_error',
                    groupbyparameters=groupbyviolin)

## Straight channel with 50% Glycein-water mixture and three curved channels with water
For Fig. 12.

Apply curve-fitting on the experimental data.

| Microchannel - Fluid | $\theta_0$ (deg) |
| :--------: | :--------: | 
| Straight - 50% Glycerin-water mixture | 82 |
| Variation 1 - water | 87 |
| Variation 2 - water | 84 | 
| Variation 3 - water | 86 | 

In [ ]:
df = pd.read_csv("Data/data_full_dataset.csv", sep=';')
index_column = ['channel-fluid', 'V']
df.set_index(index_column, inplace=True)
df.sort_index(inplace=True)

# Load separate data
#df = df.loc["straight-glycerin"]
df = df.loc["var1-water"]
#df = df.loc["var2-water"]
#df = df.loc["var3-water"]

# Static contact angle
#theta_eq = 82/180*np.pi # straight-glycerin
theta_eq = 87/180*np.pi # var1-water
#theta_eq = 84/180*np.pi # var2-water
#theta_eq = 86/180*np.pi # var3-water

rho_glycerin = 1142
mu_glycerin = 0.008026
sigma_air_glycerin = 68.12e-3

rho_water = 998 # water density in kg/m3
mu_water = 0.00100036 # dynamic viscosity in Pa*s
sigma_air_water = 72.74e-3

#optimize process for straight channel with glycerin
#mu = mu_glycerin
#sigma = sigma_air_glycerin

#optimize process for curved channels with water
mu = mu_water
sigma = sigma_air_water

def combined_opt(uCL, zeta, beta):
    return (( np.arccos(-zeta*uCL/sigma + np.cos(theta_eq)))**3 + 9*(uCL * mu / sigma)*beta)**(1/3)

def HDT_opt(uCL, beta):
    return ((theta_eq**3 + 9*(uCL * mu / sigma)*beta)**(1/3))

def MKT_opt(uCL, lamda, kappa0):
    return (arccos(cos(theta_eq) - 2*1.380649e-23*295.15 / (sigma*lamda**2) * arcsinh(uCL/(2*kappa0*lamda))))

def MKT_linear_opt(uCL, zeta):
    return (arccos(cos(theta_eq) - uCL*zeta/sigma))

popt_combined, pcov_combined = curve_fit(combined_opt, df.u_real.values*0.001, df.theta_code.values/180*np.pi, p0=[1e-4, 1e2]) #input: function, xdata, ydata, startValues
popt_MKT, pcov_MKT = curve_fit(MKT_opt, df.u_real.values*0.001, df.theta_code.values/180*np.pi, p0=[1e-09, 1e+06])#input: function, xdata, ydata, startValues
popt_HDT, pcov_HDT = curve_fit(HDT_opt, df.u_real.values*0.001, df.theta_code.values/180*np.pi, p0=1e2) 
#popt, pcov = curve_fit(MKT_opt, df1.u_real.values*0.001, df1.theta_code.values/180*np.pi, p0=[1e-09, 1e+06, 83])#input: function, xdata, ydata, 

lamda_error = (np.sqrt(pcov_MKT[0,0]))
kappa0_error = (np.sqrt(pcov_MKT[1,1]))

print(popt_VC) # these are zeta and beta
print(popt_MKT)
print(popt_HDT)

## Straight channel with 50% Glycein-water mixture and three curved channels with water
For Fig. 12.

Create and save plots.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(6, 6), dpi=300)

labels = ['straight-glycerin', 'var1-water', 'var2-water', 'var3-water']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
markers = ['o', 'X', 'P', 's']

# Free parameters obtained from curve-fitting
lamda = popt_MKT[0]
kappa0 = popt_MKT[1]

zeta = popt_combined[0]
beta_combined = popt_combined[1]

beta_HDT = popt_HDT[0]

# Index for plot cosmetics
i = 1

for value, plotdf in df.groupby(level='V'):
    mean, errors = cal_errors(plotdf['theta_code'])
    plt.errorbar(np.mean(plotdf['ca']), mean, yerr=errors, fmt=markers[i], capsize=4, color=colors[i])
    plt.xscale('log')
    plt.xlabel('Capillary number $Ca$')
    plt.ylabel(r'Dynamic contact angle $\theta_d$ [degree]')
    plt.legend()
plt.scatter([], [], marker='o', label=labels[i], color=colors[i])

    
uCL = np.linspace(0.00001, 0.033, 1000)
caCL= uCL * mu / sigma
theta_MKT = cal_theta_MKT(uCL, mu, sigma, theta_eq, lamda, kappa0)
theta_HDT = cal_theta_HDT(uCL, mu, sigma, theta_eq, beta_HDT)
theta_combined = cal_theta_combined(uCL, mu, sigma, theta_eq, zeta, beta_combined)
theta_Jiang = cal_theta_Jiang(theta_eq, caCL)
theta_Bracke = cal_theta_Bracke(theta_eq, caCL)

plt.plot([], [], ' ', label="Dynamic contact angle models")
plt.plot(caCL, theta_MKT, label= fr"MKT: $\lambda=${lamda:.2e}[m], $\kappa^0=${kappa0:.2e}[Hz]", color='k')
plt.plot(caCL, theta_HDT, label=fr"HDT: $\beta=${beta_HDT:.2e}", color='k', linestyle=linestyles_dict["densely dashdotted"])
plt.plot(caCL, theta_combined, label= fr"Combined: $\beta=${beta_VC:.2e}, $\zeta=${zeta:.2e}[Pas]", color='k', linestyle='dashdot')
plt.plot(caCL, theta_Jiang, label="Jiang", color='k', linestyle='dashed')
plt.plot(caCL, theta_Bracke, label="Bracke", color='k', linestyle='dotted')
plt.grid()

plt.legend(loc=2)
plt.savefig(os.path.join(figure_directory, labels[i]), bbox_inches="tight")

## Different parts of variation 3: straight, position 1 and position 2
For Fig. 14.

| Channel part | $\theta_0$ (deg) | $\lambda$ (nm) | $\kappa^0$ (kHz) |
| :--------: | :--------: | :--------: | :--------: |
| Inside position 1 | 70 | 1.03 | 424.12 |
| Inside position 2 | 86 | 1.20 | 195.86 |
| Outside position 1 | 86 | 1.03 | 400.75 |
| Outside position 2 | 86 | 1.02 | 410.16 |
| Straight | 86 | 1.07 | 284.86 |

In [ ]:
figure(figsize=(8, 3), dpi=300)

label_var3 = ['inside1', 'inside2', 'outside1', 'outside2', 'straight']
# Measured static contact angles of water in different parts of variation 3: inside position 1, inside position 2, outside position1, outside position 2 and straight part
theta0_var3 = [70, 86, 86, 86, 86]
# Fitting parameters lambda and kappa_0 of water  in different parts of variation 3: inside position 1, inside position 2, outside position1, outside position 2 and straight part
lambda_var3 = [1.03e-9, 1.34e-9, 1.02e-9,  1.03e-9, 1.03e-9]
kappa0_var3 = [424.12e3, 107.37e3, 457.26e3, 406.92e3, 385.61e3]

# Load the csv for variation 3
df_var3 = pd.read_csv("Data/data_variation3_part.csv", sep=';')
index_columns_var3 = ['part', 'V']
df_var3.set_index(index_columns_var3, inplace=True)
df_var3.sort_index(inplace=True)

group_by_parameters = list(index_columns_var3)
group_by_parameters.remove('V')
unique_values = list(df_var3.index.unique(level='part'))
markers, linestyles, colors = assign_styles(unique_values)

for params, subsetdf in  df_var3.groupby(level=group_by_parameters):
    
    for value, plotdf in subsetdf.groupby(level='V'):
        mean, errors = cal_errors(plotdf['theta_code'])
        plt.errorbar(np.mean(plotdf['ca']), mean, yerr=errors, fmt=markers[params], capsize=4, color=colors[params])
        plt.xscale('log')
        plt.xlabel('Capillary number $Ca$')
        plt.ylabel(r'Dynamic contact angle $\theta_d$ [degree]')
        plt.legend()
    plt.scatter([], [], marker=markers[params], label=params)
    
for i in range(len(theta0_var3)):
    
    uCL = np.linspace(0.00001, 0.023, 1000)
    plt.plot(cal_caCL(uCL, mu_water, sigma_air_water), 
             cal_theta_MKT(uCL, mu_water, sigma_air_water, theta0_var3[i]/180*np.pi, lambda_var3[i], kappa0_var3[i]), 
             label='MKT - '+label_var3[i])
    plt.legend(ncol=2)
    
plt.savefig(os.path.join(figure_directory, "variation3_part_error_bars"), bbox_inches="tight")